In [1]:
import os
import sys
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from models.mesh_classifier import ClassifierModel

from data.data import ClassificationData, collate_fn
import utils.util as util

In [2]:
dataroot = 'datasets/human_class' 
phase = 'train'
export_folder = 'export'
checkpoints_dir = 'checkpoints'
name = 'classification'
ninput_edges = 750
num_aug = 1

batch_size = 16
num_workers = 4

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
dataset = ClassificationData(dataroot=dataroot, phase=phase, export_folder=export_folder, device=device, ninput_edges=ninput_edges, num_aug=num_aug)
dataloader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    collate_fn=collate_fn
)

nclasses = dataset.get_nclasses()   
ninput_channels = dataset.get_ninput_channels()

loaded mean / std from cache


In [6]:
is_train = True

norm_type = 'group' # batch, instance, group
num_groups = 16
ncf = [64, 128, 256, 256] # convolutional filters
pool_res = [600, 450, 300, 180] # pooling resolution
fc_n = 100 # number of fully connected layer between fc and nclasses
resblocks = 3 # number of res blocks
init_type = 'normal' # normal, xavier, kaiming, orthogonal
init_gain = 0.02 # for init_type = normal, xavier, kaiming
lr = 0.001 # initial learning rate for adam
beta1 = 0.9 # momentum1 in adam

classifier_model = ClassifierModel(is_train=is_train, device=device, nclasses=nclasses, ninput_channels=ninput_channels)

NotImplementedError: normalization layer [BatchNorm2d] is not found